# Advent of Code

## 2022-012-014
## 2022 014

https://adventofcode.com/2022/day/14

In [2]:
# Parse the input to set up the grid for simulation

def parse_input(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    rock_paths = []
    for line in lines:
        path = line.strip().split(" -> ")
        rock_paths.append([tuple(map(int, point.split(','))) for point in path])
    
    return rock_paths

def build_cave(rock_paths):
    # Determine the size of the grid
    max_x = max(point[0] for path in rock_paths for point in path)
    max_y = max(point[1] for path in rock_paths for point in path)
    min_x = min(point[0] for path in rock_paths for point in path)

    offset_x = min_x  # Offset to align grid
    width = max_x - min_x + 1
    height = max_y + 1

    # Initialize the grid
    grid = [['.' for _ in range(width)] for _ in range(height)]

    # Mark rocks
    for path in rock_paths:
        for i in range(len(path) - 1):
            x1, y1 = path[i]
            x2, y2 = path[i + 1]
            if x1 == x2:  # Vertical line
                for y in range(min(y1, y2), max(y1, y2) + 1):
                    grid[y][x1 - offset_x] = '#'
            elif y1 == y2:  # Horizontal line
                for x in range(min(x1, x2), max(x1, x2) + 1):
                    grid[y1][x - offset_x] = '#'

    return grid, offset_x

def simulate_sand(grid, offset_x):
    sand_source = (500 - offset_x, 0)  # Adjust for offset
    sand_count = 0
    width = len(grid[0])
    height = len(grid)

    while True:
        x, y = sand_source
        while True:
            if y + 1 >= height:  # Abyss below
                return sand_count
            if grid[y + 1][x] == '.':  # Down
                y += 1
            elif x > 0 and grid[y + 1][x - 1] == '.':  # Down-left
                y += 1
                x -= 1
            elif x < width - 1 and grid[y + 1][x + 1] == '.':  # Down-right
                y += 1
                x += 1
            else:  # Comes to rest
                grid[y][x] = 'o'
                sand_count += 1
                break

# Input file path
file_path = 'input.txt'

# Parse the input and build the cave
rock_paths = parse_input(file_path)
grid, offset_x = build_cave(rock_paths)

# Simulate the sand falling and get the result
sand_count = simulate_sand(grid, offset_x)
sand_count

1133

In [3]:
def simulate_sand_with_floor(grid, offset_x):
    # Add a floor at y = max_y + 2
    floor_y = len(grid) + 1
    width = len(grid[0])
    grid.append(['.'] * width)  # Add the floor row as air initially
    grid.append(['#'] * width)  # Add the floor row as solid rock

    sand_source = (500 - offset_x, 0)  # Adjust for offset
    sand_count = 0

    while grid[sand_source[1]][sand_source[0]] == '.':  # Stop when the source is blocked
        x, y = sand_source
        while True:
            if grid[y + 1][x] == '.':  # Down
                y += 1
            elif x > 0 and grid[y + 1][x - 1] == '.':  # Down-left
                y += 1
                x -= 1
            elif x < width - 1 and grid[y + 1][x + 1] == '.':  # Down-right
                y += 1
                x += 1
            else:  # Comes to rest
                grid[y][x] = 'o'
                sand_count += 1
                break

    return sand_count

# Rebuild the cave and simulate with the floor added
grid, offset_x = build_cave(rock_paths)
sand_count_with_floor = simulate_sand_with_floor(grid, offset_x)
sand_count_with_floor

6712

In [4]:
def parse_input(lines):
    """Parse the input lines into a set of rock cells."""
    rocks = set()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        points = [tuple(map(int, p.split(','))) for p in line.split('->')]
        # Draw lines between consecutive points
        for i in range(len(points) - 1):
            x1, y1 = points[i]
            x2, y2 = points[i+1]
            if x1 == x2:
                # Vertical line
                start, end = sorted([y1, y2])
                for y in range(start, end + 1):
                    rocks.add((x1, y))
            elif y1 == y2:
                # Horizontal line
                start, end = sorted([x1, x2])
                for x in range(start, end + 1):
                    rocks.add((x, y1))
            else:
                # Shouldn't happen in AoC day 14 input (only straight lines)
                raise ValueError("Input line not horizontal or vertical.")
    return rocks


def simulate_sand(rocks):
    """Simulate the sand until it flows into the abyss."""
    # Find the maximum y of any rock - going beyond this means falling into abyss
    max_y = max(y for _, y in rocks)
    sand = set()
    source = (500, 0)

    # Directions sand tries to move in order:
    # down, down-left, down-right
    directions = [(0, 1), (-1, 1), (1, 1)]

    units_resting = 0

    while True:
        sx, sy = source
        while True:
            # If sand moves beyond max_y, it falls forever - end simulation
            if sy > max_y:
                return units_resting

            # Try to move the sand
            moved = False
            for dx, dy in directions:
                nx, ny = sx + dx, sy + dy
                if (nx, ny) not in rocks and (nx, ny) not in sand:
                    # Move the sand to the new position
                    sx, sy = nx, ny
                    moved = True
                    break

            if not moved:
                # Sand comes to rest
                sand.add((sx, sy))
                units_resting += 1
                break


if __name__ == "__main__":
    # Read input
    with open("input.txt") as f:
        lines = f.readlines()

    rocks = parse_input(lines)
    result = simulate_sand(rocks)
    print(result)

1133


In [5]:
def parse_input(lines):
    """Parse the input lines into a set of rock cells."""
    rocks = set()
    for line in lines:
        line = line.strip()
        if not line:
            continue
        points = [tuple(map(int, p.split(','))) for p in line.split('->')]
        for i in range(len(points) - 1):
            x1, y1 = points[i]
            x2, y2 = points[i+1]
            if x1 == x2:
                # Vertical line of rocks
                start, end = sorted([y1, y2])
                for y in range(start, end + 1):
                    rocks.add((x1, y))
            elif y1 == y2:
                # Horizontal line of rocks
                start, end = sorted([x1, x2])
                for x in range(start, end + 1):
                    rocks.add((x, y1))
            else:
                raise ValueError("Input line is not purely horizontal or vertical.")
    return rocks

def simulate_sand_with_floor(rocks):
    max_y = max(y for _, y in rocks)
    floor_y = max_y + 2

    sand = set()
    source = (500, 0)
    directions = [(0, 1), (-1, 1), (1, 1)]
    units_resting = 0

    while True:
        # If the source itself is filled with sand, stop
        if source in sand:
            return units_resting

        sx, sy = source
        while True:
            moved = False
            for dx, dy in directions:
                nx, ny = sx + dx, sy + dy
                # Check if next position is blocked by rock, sand, or the floor
                if ny == floor_y:  # This simulates the infinite floor
                    continue
                if (nx, ny) not in rocks and (nx, ny) not in sand:
                    sx, sy = nx, ny
                    moved = True
                    break

            if not moved:
                # Sand comes to rest
                sand.add((sx, sy))
                units_resting += 1
                break

if __name__ == "__main__":
    with open("input.txt") as f:
        lines = f.readlines()

    rocks = parse_input(lines)
    result = simulate_sand_with_floor(rocks)
    print(result)

27566
